<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigdataII_01_folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium==0.12.1
!pip install geocoder


# データセット

データカタログ
* https://www.data.go.jp/?lang=ja

Google Dataset
* https://datasetsearch.research.google.com

Kaggle
* https://www.kaggle.com/datasets

github 上で公開されているデータセットのリンク集
* https://github.com/arXivTimes/arXivTimes/tree/master/datasets
* https://github.com/awesomedata/awesome-public-datasets

国勢調査サイト(e-stat)
* https://www.e-stat.go.jp/
* 都道府県別の人口を検索
* 平成29年10月1日現在人口推計 都道府県，男女別人口－総人口

岡山オープンデータカタログ
* http://www.okayama-opendata.jp/opendata/index.action



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

%cd '/content/drive/MyDrive/Lecture_BigData'

In [ ]:
import pandas as pd
import folium
from folium import plugins
print(folium.__version__)

In [ ]:
# 国勢調査サイト　⇒ 「人口推計」検索　⇒　表示項目変更　⇒　ダウンロード

population_df = pd.read_csv("population.csv",encoding="shift-JIS")
population_df.head()

# 時系列ヒートマップ表示

In [ ]:
import geocoder

# 日本の 緯度・経度 を取得
japan = geocoder.arcgis("日本")
japan = japan.latlng
 
# 都道府県名抽出
prefecture = population_df["全国・都道府県"].tolist()

# 都道府県の 緯度・経度 リストを作成
prefecture_latlng = []
for p in prefecture:
    g = geocoder.arcgis(p)
    prefecture_latlng.append(g.latlng)

geo_df = pd.DataFrame([i for i in range(1, len(prefecture)+1)], columns = ["id"])
geo_df["name"] = pd.DataFrame(prefecture)
geo_df["lat"] = pd.DataFrame([r[0] for r in prefecture_latlng])
geo_df["lng"] = pd.DataFrame([r[1] for r in prefecture_latlng])

In [ ]:
# フォーマットを揃える

population_df["id"] = population_df.index+1
prefecture_df = pd.merge(population_df, geo_df, on=["id"])
prefecture_df = prefecture_df.drop(["name","id"],axis=1)
prefecture_df.head()

In [ ]:
time_series = ["15","16","17","18","19"]

for year in time_series:
    date = "20"+year+"年"
    temp_df = prefecture_df[date].apply(lambda d: pd.to_numeric(d.replace(',',''), downcast='integer'))
    prefecture_df[date] = temp_df / max(temp_df)
    
prefecture_df.head()

In [ ]:
heat_data = [[[row['lat'],row['lng'], row["20"+year+"年"]] for index, row in prefecture_df.iterrows()] for year in time_series]

In [ ]:
japan_map = folium.Map(
    location=[35, 135],
    width=720,
    height=540,
    zoom_start=5
)

hm = plugins.HeatMapWithTime(
    heat_data,
    index=["20"+year+"年" for year in time_series],
    auto_play=False,
    radius=40,
    max_opacity=1,
    gradient={0.1: 'blue', 0.25: 'lime', 0.5:'yellow',0.75: 'red'}
)

hm.add_to(japan_map)

#for i, r in prefecture_df.iterrows():
#    popup=folium.Popup(r["全国・都道府県"], max_width=10, show=False)
#    folium.Marker(location=[r["lat"], r["lng"]], popup=popup).add_to(japan_map)

japan_map

In [ ]:
from folium.plugins import HeatMap

# 日本のマップオブジェクトを作成
map = folium.Map(japan, width=800, height=600, zoom_start=5)
 
# ヒートマップの表示
HeatMap(prefecture_latlng).add_to(map)
 
# htmlを作成
map

In [ ]:
import folium
import pandas as pd

sapporo_cities = pd.DataFrame({
    'city': ['ハルタ', 'ブルクベーカリー 札幌円山本店', 'DONQ 円山店', 'すぎうらベーカリー 円山店','ペンギンベーカリーカフェ 円山裏参道店'],
    'latitude': [43.057320, 43.056397, 43.055939, 43.056211, 43.054867],
    'longtude': [141.322706, 141.321842, 141.319576, 141.318643,141.321690],
    'population': [1000, 1000, 1000, 1000, 1000]
})

maruyama_map = folium.Map(
    location=[43.056000, 141.321000],
    zoom_start=15
)

for i, r in sapporo_cities.iterrows():
    
    popup = folium.Popup(
        r['city'],
        max_width=1000,
        show=True
    )

    marker = folium.Marker(
        location=[r['latitude'], r['longtude']],
        popup=popup
    )
    
    marker.add_to(maruyama_map)

maruyama_map

# Dcument

https://python-visualization.github.io/folium/

## Map

tiles option

検索する

https://maps.gsi.go.jp/development/ichiran.html

http://leaflet-extras.github.io/leaflet-providers/preview/

## Marker

tooltip

folim.Icon()

color, icon_color

icon 
https://glyphsearch.com/?library=glyphicons

In [ ]:
#--- code here---#


## PolyLine

In [ ]:
!pip install selenium

データ準備

In [ ]:
# あとで使うライブラリも併せて読み込み
import os
import folium
import pandas as pd
from time import sleep as slp
from selenium import webdriver
import glob
from selenium.webdriver.chrome.options import Options

# csvをpandas dataframeに保存
data = pd.read_csv("dataset_temp/gps.csv",encoding="utf-8")

data.head()

In [ ]:
data[data["人"] == "B"].head()

In [ ]:
# 今回はA,Bの人しか居ないが、
# 一応何人来てもいいように人のリストを作り、
# それをforループで回す
person_list = data["人"].unique()

map 設定

PolyLine 設定

In [ ]:
# 地図に表示する色
# foliumでサポートしているのは下記19色
color_list=['red','blue','green','purple','orange','darkred','lightred','beige','darkblue','darkgreen','cadetblue','darkpurple','white','pink','lightblue','lightgreen','gray','black','lightgray']

# 地図オブジェクトを作成
m = folium.Map(tiles='OpenStreetMap', zoom_start=10)

for idx,person in enumerate(person_list):
    # 一人分のデータだけをdata_tempに格納する
    data_temp = data[data["人"] == person]
    
    # data_tempの順番を日時で昇順ソート
    data_temp = data_temp.sort_values('日時', ascending=True)
    
    # data_tempの緯度経度だけを
    data_temp_lat_lon = data_temp[["緯度","経度"]]
    
    # 緯度経度を配列に格納
    locs = data_temp_lat_lon.values
    
    # 色を指定
    line_color = color_list[idx%len(color_list)]
    
    # 地図に線を追加する。緯度経度の配列をそのまま線として使う
    folium.PolyLine(
        locs,
        color=line_color,
        popup=person
    ).add_to(m)
    
# 地図の表示範囲を緯度経度の最低最大とする
m.fit_bounds([[data["緯度"].min(),data["経度"].min()], [data["緯度"].max(),data["経度"].max()]])

# 地図を表示する
m

html に保存する場合

In [ ]:
for person in person_list:
    # 一人分のデータだけをdata_tempに格納する
    data_temp = data[data["人"] == person]
    
    # data_tempの順番を日時で昇順ソート
    data_temp = data_temp.sort_values('日時', ascending=True)
    
    # data_tempの緯度経度だけをデータフレームに残す
    data_temp_lat_lon = data_temp[["緯度","経度"]]
    
    # 地図オブジェクトを生成
    m = folium.Map(tiles='OpenStreetMap')
    
    # 緯度経度を配列に格納
    locs = data_temp_lat_lon.values
    
    # 地図に線を追加する。緯度経度の配列をそのまま線として使う
    folium.PolyLine(locs).add_to(m)
    
    # 地図の表示範囲を緯度経度の最低最大とする
    m.fit_bounds([[data_temp["緯度"].min(),data_temp["経度"].min()], [data_temp["緯度"].max(),data_temp["経度"].max()]])
    
    # htmlに保存する
    # ./html/xx.htmlに保存される
    m.save(outfile="dataset_temp/"+person+".html")